In [ ]:
import h5py
import numpy as np

f = h5py.File("data/train_digitStruct.mat", "r")
bbox_refs = f["digitStruct"]["bbox"]
names = f["digitStruct"]["name"]

def read_attr(attr):
    """Read a bbox attribute (left/top/width/height/label)."""
    # If it's a single reference
    if len(attr) == 1:
        return attr[0][0]
    # If multiple references
    return np.array([a[0] for a in attr]).astype(float)

def get_boxes(i):
    # Dereference the bbox for image i
    bboxes = bbox_refs[i]

    def get_field(bbox, field):
        val = f[bbox][field][0][0]
        return val;

    dataDict = {}
    i = 0
    for bbox in bboxes:
        dataDict[i]['left'] = get_field(bbox, "left")
        dataDict[i]['top'] = get_field(bbox, "top")
        dataDict[i]['height'] = get_field(bbox, "height")
        dataDict[i]['width'] = get_field(bbox, "width")
        dataDict[i]['label'] = get_field(bbox, "label")
        i += 1

    return dataDict

def read_string(ref):
    return "".join([chr(c[0]) for c in f[ref]])

def get_name(i):
    name_ref = names[i][0]
    return read_string(name_ref)

# Example: first image
filename = get_name(992)
box_info = get_boxes(992)
print(filename, box_info)


bboxes = f['digitStruct']['bbox'][993]
print(bboxes.shape)
print(bboxes.dtype)


# list(bboxes)

for ref in bboxes:
    print(f[ref]['left'][0][0])
    print(list(f[ref]['top']))
    print(list(f[ref]['width']))
    print(list(f[ref]['height']))
    print(list(f[ref]['label']))
# list(f[ref]['left'])




